In [1]:
from google.cloud import bigquery
import pandas as pd

In [3]:
# Set up a client
key_path = "../credentials/dbt_service_account.json"
client = bigquery.Client.from_service_account_json(key_path)

In [13]:
# Check available date ranges
date_query = """
SELECT
    MIN(PARSE_DATE('%Y%m%d', _TABLE_SUFFIX)) AS first_date,
    MAX(PARSE_DATE('%Y%m%d', _TABLE_SUFFIX)) AS last_date
FROM 
    `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
"""
dates = client.query(date_query).to_dataframe()
dates

/Users/mariroci/Library/Caches/pypoetry/virtualenvs/onboarding-funnel-analysis-iGMZaZzf-py3.13/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,first_date,last_date
0,2020-11-01,2021-01-31


Dates available between 2020-11-01 and 2021-01-31. Choosing a 2 month period between 2020-11-01 and 2020-12-31.

In [14]:
# List distinct events and their counts
sql = """
SELECT
  event_name,
  COUNT(*) AS hits
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20201231'   -- 2-month window
GROUP BY event_name
ORDER BY hits DESC
"""
events_df = client.query(sql).to_dataframe()
events_df.head(15)


/Users/mariroci/Library/Caches/pypoetry/virtualenvs/onboarding-funnel-analysis-iGMZaZzf-py3.13/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,event_name,hits
0,page_view,931424
1,user_engagement,808624
2,scroll,354075
3,view_item,299097
4,session_start,238421
5,first_visit,168589
6,view_promotion,136219
7,add_to_cart,43021
8,begin_checkout,27723
9,select_item,20778


In [16]:
events_df.event_name.to_list()

['page_view',
 'user_engagement',
 'scroll',
 'view_item',
 'session_start',
 'first_visit',
 'view_promotion',
 'add_to_cart',
 'begin_checkout',
 'select_item',
 'view_search_results',
 'add_shipping_info',
 'add_payment_info',
 'select_promotion',
 'purchase',
 'click',
 'view_item_list']